In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/*Utad/3ºAño/IA/Practicas_IA/Practica_2/intervenciones.zip

Archive:  /content/drive/MyDrive/*Utad/3ºAño/IA/Practicas_IA/Practica_2/intervenciones.zip
  inflating: intervencionesAbascal.txt  
  inflating: intervencionesCasado.txt  
  inflating: intervencionesSanchez.txt  


# Notebook 699: Tarea calificada 2, INAR 23-24

## Generación de texto seq2seq model
## A partir de textos de parlamentarios españoles (anteriores a 2022)

## Nota importante

Esta tarea en su versión 2023-24 surge del excelente trabajo de varios compañeros del curso 2022-23, que aunque yo proporcioné un dataset de textos a partir de las intervenciones de parlamentarios (los líderes de varios partidos en 2021-22, alguno de los cuales ya no está en la política española), hicieron un extraordinario "escrapeo" de la web del Congreso de los Diputados y enriquecieron de forma notable el dataset. Este es el que propongo para esta tarea.

Debo decir que si hay un texto (o lenguaje natural) libre de derechos y especialmente actual, son las intervenciones (estrictamente **públicas**) de los representantes elegidos en elecciones, y que el Congreso debería facilitar, no ya para su uso en estas tareas, sino para cualquier estudioso del español, o de la política, o de la psicología de los políticos.

Por supuesto, esto son opiniones estrictamete mías, en el momento concreto en que las escribo, y sencillamente quiero hacer homenaje a los que colaboraron tanto con este trabajo que espero encontréis interesante.

## ¿De qué trata esta tarea?

Pues ni más ni menos que de generar texto en español a partir de texto de parlamentarios, basado en el tutorial que hemos seguido en clase:

https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419

Para facilitar la tarea se propone un pre-proceso (basado en la tarea 2021-22), y la tarea se concreta en el modelo para generar texto y en las pruebas de la calidad del texto generado.


## Calificación

Está explicada en la entrada correspondiente de Blackboard. Básicamente, hay un mínimo que consiste en proponer tres modelos de red recurrente, uno para cada parlamentario, entrenarlos, y **evaluarlos** generando texto y comentando su calidad.

Para llegar a la máxima nota, propongo poner a dialogar los tres modelos.

Pero por supuesto, valoraré el trabajo de construcción del modelo. Para esta tarea no hay una "medida" como la accuracy en la tarea 1. Será relativamente subjetiva. Por eso parece aconsejable comenzar con modelos pequeños o con pocas etapas e ir refinando.

## Setup

Para facilitar la tarea propongo unas cuantas casillas para cargar en memoria los textos, tres .txt que están incluidos en un .zip.

## Nota importante

La codificación (juego de caracteres) es UTF-8 y creo que debe seguir siendo así. *NO* abráis los .txt con el Notepad de Windows, sino con el Notepad+++ que os permitiría cambiarlo o devolverlo a UTF-8 (o Unicode si queréis).

A pesar que la salida por pantalla (en mi sistema, un Linux) de caracteres ñ y acentuados parece que está mal, luego la generación de texto (insisto, lo he comprobado en mi sistema) es correcta en español.


### Import TensorFlow and other libraries

In [4]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import os


## Lectura de ficheros de datos

In [5]:
datos_abascal   = "intervencionesAbascal.txt"
datos_sanchez   = "intervencionesSanchez.txt"
datos_casado    = "intervencionesCasado.txt"

### Read the data

First, look in the text:

In [6]:
# Read, then decode for py2 compat.
text = open(datos_sanchez, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 239623 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])

Muchas gracias, señora presidenta. Claro que no, no hemos renunciado, señor Casado, ni a crear empleo ni a reducir el paro ni tampoco a eliminar, por ejemplo, el despido por baja médica que recogía la reforma laboral del Partido Popular y que ayer pr


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

104 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [9]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [11]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[51, 52, 53, 54, 55, 56, 57], [74, 75, 76]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [12]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [13]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [15]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


# Fases propuestas para la elaboración del modelo

### 1. Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(239623,), dtype=int64, numpy=array([37, 71, 53, ..., 69, 11,  2])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

M
u
c
h
a
s
 
g
r
a


In [19]:
seq_length = 100

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'M' b'u' b'c' b'h' b'a' b's' b' ' b'g' b'r' b'a' b'c' b'i' b'a' b's'
 b',' b' ' b's' b'e' b'\xc3\xb1' b'o' b'r' b'a' b' ' b'p' b'r' b'e' b's'
 b'i' b'd' b'e' b'n' b't' b'a' b'.' b' ' b'C' b'l' b'a' b'r' b'o' b' '
 b'q' b'u' b'e' b' ' b'n' b'o' b',' b' ' b'n' b'o' b' ' b'h' b'e' b'm'
 b'o' b's' b' ' b'r' b'e' b'n' b'u' b'n' b'c' b'i' b'a' b'd' b'o' b','
 b' ' b's' b'e' b'\xc3\xb1' b'o' b'r' b' ' b'C' b'a' b's' b'a' b'd' b'o'
 b',' b' ' b'n' b'i' b' ' b'a' b' ' b'c' b'r' b'e' b'a' b'r' b' ' b'e'
 b'm' b'p' b'l' b'e' b'o'], shape=(101,), dtype=string)


In [21]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Muchas gracias, se\xc3\xb1ora presidenta. Claro que no, no hemos renunciado, se\xc3\xb1or Casado, ni a crear empleo'
b' ni a reducir el paro ni tampoco a eliminar, por ejemplo, el despido por baja m\xc3\xa9dica que recog\xc3\xada la r'
b'eforma laboral del Partido Popular y que ayer precisamente derogamos en el Consejo de Ministros.\nMuch'
b'as gracias, se\xc3\xb1ora presidenta. Gracias, se\xc3\xb1ora Arrimadas. Simplemente, quiero decirle que este Gobier'
b'no pone las instituciones p\xc3\xbablicas al servicio de los espa\xc3\xb1oles no por una cuesti\xc3\xb3n de convicci\xc3\xb3n, si'


In [22]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [23]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Muchas gracias, se\xc3\xb1ora presidenta. Claro que no, no hemos renunciado, se\xc3\xb1or Casado, ni a crear emple'
Target: b'uchas gracias, se\xc3\xb1ora presidenta. Claro que no, no hemos renunciado, se\xc3\xb1or Casado, ni a crear empleo'


### 2. Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## 3. Build The Model

Puedes usar cualquiera de los modelos (RNN, LSTM, GRU) que hemos visto en clase. Por supuesto, del tamaño del modelo (capas, neuronas en cada capa) así como de las épocas (más adelante) dependerá el tiempo de proceso en el .fit

In [27]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [28]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [29]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 105) # (batch_size, sequence_length, vocab_size)


## 4. Summary y fit del modelo



Aconsejable el uso de GPU en Google Colab, aunque yo he conseguido hacer 200 épocas de mi modelo en un tiempo razonable (menos de 1 hora).

**RECUERDA GRABAR EL MODELO ENTRENADO PARA PODER REUTILIZARLO POSTERIORMENTE**

In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  26880     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  107625    
                                                                 
Total params: 4072809 (15.54 MB)
Trainable params: 4072809 (15.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [33]:
sampled_indices

array([100,  52,  26,  18,  22, 104,  63,  31,  97,  13,  81,  52,  66,
        57,  96,  28,  99,  61,  99,   3,  47,  61,  68,   2,  91,  29,
        83,  68,  90,  90,  75,  17,  32,  94,  36,  34,  51,   8,  31,
        15,  56,  58,  48,   1,  48,  37, 104,  36,  45,  86,  76,  68,
        15,  38, 101,  63,  94,  40,  65,  82,  38,  48,  17, 103,  80,
        35,  38,  48,  34,  60,  48,  80,   4,  11,  37,  25,  11,  67,
        12,  50,  37,  30,  31,  57,   9,  58,  25, 102, 104,  58, 100,
        34,  35,  30,  14,  84,  36,  29,  16,   2])

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ca una sociedad con pactos y una sociedad sin pactos. Una sociedad sin pactos es lo que hab\xc3\xada antes '

Next Char Predictions:
 b'\xe2\x80\x94bB6:\xe2\x80\xa6mG\xcc\x831\xc3\x80bpg\xcc\x81D\xe2\x80\x8ck\xe2\x80\x8c Wkr\n\xc3\xadE\xc3\x89r\xc3\xa9\xc3\xa9y5H\xc3\xbaLJa)G3fhX\x07XM\xe2\x80\xa6LU\xc3\x93zr3N\xe2\x80\x95m\xc3\xbaPo\xc3\x81NX5\xe2\x80\x99\xc2\xbfKNXJjX\xc2\xbf!.MA.q0ZMFGg,hA\xe2\x80\x98\xe2\x80\xa6h\xe2\x80\x94JKF2\xc3\x8dLE4\n'


In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 105)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.650911, shape=(), dtype=float32)


In [37]:
tf.exp(example_batch_mean_loss).numpy()

104.68029

In [38]:
model.compile(optimizer='adam', loss=loss)

In [39]:

'''
import shutil
import os

# Directorio que acabamos de crear
directory_to_delete = './training_checkpoints'

# Eliminar el directorio
shutil.rmtree(directory_to_delete)
'''

"\nimport shutil\nimport os\n\n# Directorio que acabamos de crear\ndirectory_to_delete = './training_checkpoints'\n\n# Eliminar el directorio\nshutil.rmtree(directory_to_delete)\n"

In [40]:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    period=10  # Guarda el modelo cada 10 época
    )
    # Dice que period esta deprecado pero si lo hago con save_freq no me funciona

In [41]:
EPOCHS = 400
 # en la época 47  loss 0.0838

In [42]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/400
37/37 [==============================] - 5s 66ms/step - loss: 3.6884
Epoch 2/400
37/37 [==============================] - 2s 49ms/step - loss: 2.6124
Epoch 3/400
37/37 [==============================] - 3s 53ms/step - loss: 2.2901
Epoch 4/400
37/37 [==============================] - 2s 53ms/step - loss: 2.1219
Epoch 5/400
37/37 [==============================] - 2s 50ms/step - loss: 1.9827
Epoch 6/400
37/37 [==============================] - 2s 48ms/step - loss: 1.8443
Epoch 7/400
37/37 [==============================] - 2s 52ms/step - loss: 1.7040
Epoch 8/400
37/37 [==============================] - 3s 50ms/step - loss: 1.5733
Epoch 9/400
37/37 [==============================] - 2s 49ms/step - loss: 1.4599
Epoch 10/400
37/37 [==============================] - 2s 54ms/step - loss: 1.3603
Epoch 11/400
37/37 [==============================] - 2s 49ms/step - loss: 1.2770
Epoch 12/400
37/37 [==============================] - 2s 49ms/step - loss: 1.2033
Epoch 13/400
37/37 [=====

In [43]:
# Acceder a las métricas de entrenamiento
train_loss = history.history['loss']

# Encontrar la época con la pérdida más baja
min_loss_epoch = train_loss.index(min(train_loss)) + 1  # Sumar 1 porque las épocas comienzan desde 1

# Imprimir la época con la pérdida más baja
print(f"Época con la pérdida más baja: {min_loss_epoch}, Loss: {min(train_loss)}")

Época con la pérdida más baja: 248, Loss: 0.057004414498806


## 5. Genera texto y evalúa su calidad

In [44]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [45]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [46]:
start = time.time()
states = None
next_char = tf.constant(['Diputado'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Diputado, esta es la respuesta que hemos dado con los instrumento que tenemos por delante. Lo hicimos así en el ámbito del diálogo social, precisamente, habla de la Guardia Civil y de la guerra valor, por hacer una apoya al Gobierno de España, pero me gustaría que asumiera un papel de oposición útil con la ciencia — consejo de la unidad de nuestros hijos y de nuestras hijas. Es decir, interpretamos la educación fue un Gobierno que teníamos en nuestro país, no ya antes de la pandemia sino antes de la crisis financiera. Con la crisis del COVID es que ilegal sostenible que nos hemos dado un sus pactos con el Gobierno va a que, por cierto, con lo de mitad política que reforzar y es también con el apoyo de integritos que se defiende es una agenda lunátación laboral del Partido Popular contar con la ultraderecha, cuenta el progreso y para la cuestión del sistema de financiación. La responsabilidad colectiva. Tengo que decirle que el pasado 14 de febrero es la seguridad española. Voy a cumpli

###Seguir

In [47]:
#!zip -r /content/sanchez.zip /content/training_checkpoints

## 6. Trabajo adicional

Por ejempl, poner en cadena los tres modelos para que "dialoguen" entre sí

In [48]:
tf.saved_model.save(one_step_model, 'one_step_sanchez')
